# Naive Bayes és Logisztikus regresszió alkalmazása multiklasszifkációs problémára

## Feladat rövid leírása
A feladat során a stack-overflow-data-small.csv felhasználásával mutatom be a Naive Bayes és logisztikus regresszió alkalmazását. 

A megfelelő adatisztási feladatok elvégzésvel a nagyobb méretű adathalmazon:
- Naive Bayes: 0.73683
- Logisztikus regresszió: 0.793

A kisebb méretű adathalmazon:
- Naive Bayes: 0.706667
- Logisztikus regresszió: 0.76958333

pontosságot sikerült elérnem.

## Adatok előkészítése

### Importok

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')
from sklearn.model_selection import train_test_split
#Bayes
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score, classification_report
#Logstic regression
from sklearn.linear_model import LogisticRegression

### Nyers adat beolvasása

In [ ]:
df = pd.read_csv('../data/stack-overflow-data-small.csv')
df = df[pd.notnull(df['tags'])]
df.head(10)

##### Szövegtisztítás előtti szavak száma

In [ ]:
df['post'].apply(lambda x: len(x.split(' '))).sum()

### Oszlopdiagram az adatokban lévő osztályok eloszlásáról
Az eredeteti adathalmazomban az egyes csoportok egyenlő számban tartalmaztak elemeket az osztályokhoz, azaz az adathalmaz jól kiegyensúlyozottnak tekinthető.  
A kisméretű fájl esetén kis eltérés van az adott adathalmaz osztályainak számában, de még közel hasonlóak, így az adathalmaz elfogadhatóan kiegyensúlyozottnak tekinthető.

In [ ]:
my_tags = ['java','html','asp.net','c#','ruby-on-rails','jquery','mysql','php','ios','javascript',
           'python','c','css','android','iphone','sql','objective-c','c++','angularjs','.net']
plt.figure(figsize=(15,5))
df.tags.value_counts().plot(kind='bar')

### Szövegtisztítás

##### Szövegellenőrzés, melyből megállapítható, hogy miket kell eltávolítani(tisztítani)

In [ ]:
def text_show(index):
    row = df[df.index == index][['post', 'tags']].values[0]
    if len(row) > 0:
        print(row[0])
        print('Tag:', row[1])

In [ ]:
text_show(5)

### Szövegtisztítás:
- html tagek
- kisbetűssé alakítás
- rossz szimbólumok törlése
- egyéb szimbólumok helyettesítése space-szel
- stopwordok eltávolítása

In [ ]:
replaceWithSpace = re.compile('[/(){}\[\]\|@,;]') #re: reguláris kifejezés
BadSymbols = re.compile('[^0-9a-z #+_]') # ^:ami nincs benne a megadott tartományban
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = BeautifulSoup(text, "html.parser").text 
    text = text.lower()
    text = BadSymbols.sub('', text)
    text = replaceWithSpace.sub(' ', text) 
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    return text

df['post'] = df['post'].apply(clean_text)

##### Tisztított szöveg eredmény

In [ ]:
text_show(5)

##### A tisztítás után megmaradt szavak száma

In [ ]:
df['post'].apply(lambda x: len(x.split(' '))).sum()

### Az adatok train, és teszt adathalmazra való bontása

In [ ]:
X = df.post
y = df.tags
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

# Miért is jó a Scikit-Learn(Sklearn) könyvtár:
#### Az Sklearn az egyik legrobosztusabb, és leghasznosabb gépi tanulási könyvtár. 
- Hatékony eszközöket kínál a statisztikai modellezéshez, beleértve az osztályozást, a regressziót, a klaszterezést és a dimenziócsökkentést is, ezért ebben a feladatban is a Sklearnt használom.

## Naive Bayes

##### https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html

In [ ]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

## Logisztikus regresszió

##### https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

### A legjobb solver megtalálása
Erre azért van szükség, mert az adathalmazhoz fontos megtalálni a leggyorsabb, és legköltséghatékonyabb solvert. Maga a solver azt jelentené, hogy milyen technikát alkalmazunk a logisztikus regresszió megvalósításához.

#### Alapvetően a dokumentáció a következő ajánlásokat teszi:
- For small datasets, ‘liblinear’ is a good choice, whereas ‘sag’ and ‘saga’ are faster for large ones;
- For multiclass problems, only ‘newton-cg’, ‘sag’, ‘saga’ and ‘lbfgs’ handle multinomial loss;
- ‘liblinear’ is limited to one-versus-rest schemes.

### Időteszt
A multiclassifikációs problémáknál ‘newton-cg’, ‘sag’, ‘saga’ ‘lbfgs’ solver tudja kezelni a veszteséget.
Ezért ezen solverekre tesztelem, hogy melyik lenne a legopcionálisabb:

In [ ]:
import time
# Solvers
solvers = ['sag', 'saga', 'newton-cg', 'lbfgs']

for sol in solvers: 
    start = time.time()
    logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=2, C=1, solver=sol)),
                ])
    logreg.fit(X_train, y_train)
    end = time.time()
    print(sol + " Fit Time: ",end-start)

Az eredmény magyarázata:
- sag, saga: 
    - A nagymennyiségű adathoz ajánlott.
    - a sag továbbfejlesztése a saga.
- newton-cg: 
    - Hesse-mátrixot használ:
       - egy többváltozós valós függvény másodrendű parciális deriváltjaiból alkot négyzetes mátrixot.
    - Éppen a másodrendű deriválás miatt lassú nagymennyiségű adatnál a solver.
- lbfgs:
    - Limited-memory Broyden–Fletcher–Goldfarb–Shanno: Az utolsó pár értéket tárolja, a memóriaspórólás miatt.
    - Gradiens kiértékelésekkel közelíti a második derivált mátrix frissítéseket. => lassú.


In [ ]:
logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=2, C=1, solver='sag')),
               ])
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))